In [ ]:
# ---------------------------------------------------------------
#   ML PIPELINE FOR WINE QUALITY (COMBINED FULL + TUNED MODELS)
#   WITH FULL EDA SECTION INSERTED
# ---------------------------------------------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import (
    train_test_split, KFold, cross_val_score,
    RandomizedSearchCV
)
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.ensemble import (
    ExtraTreesClassifier,
    RandomForestClassifier,
    GradientBoostingClassifier
)

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore")

In [ ]:
# ---------------------------------------------------------------
# 1. DATA LOADING
# ---------------------------------------------------------------

print("\n--- 1. DATA LOADING & EXPLORATION ---")

try:
    data = pd.read_csv("/content/WineQT.csv")
except FileNotFoundError:
    print("Error: WineQT.csv not found.")
    exit()

print(f"Shape: {data.shape}")
print(data.head())
print("Missing Values:\n", data.isnull().sum())

df = data.copy()  # For EDA usage consistency


--- 1. DATA LOADING & EXPLORATION ---
Error: WineQT.csv not found.


NameError: name 'data' is not defined

: 

In [ ]:

# ---------------------------------------------------------------
# 1B. FULL EDA SECTION (YOUR NEWLY ADDED CODE)
# ---------------------------------------------------------------

print("\n--- 1B. EXPLORATORY DATA ANALYSIS (EDA) ---")

In [ ]:

# =============================
# HISTOGRAMS WITH KDE
# =============================
num_features = df.columns.drop("quality")

plt.figure(figsize=(15, 25))
for i, feature in enumerate(num_features):
    plt.subplot(len(num_features)//3 + 1, 3, i+1)
    sns.histplot(df[feature], kde=True)
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Density')
    plt.tight_layout()
plt.show()

print("\nHistograms with KDE generated for all features.")


In [ ]:

# =============================
# BOXPLOTS FOR OUTLIERS
# =============================
plt.figure(figsize=(15, 25))
for i, feature in enumerate(num_features):
    plt.subplot(len(num_features)//3 + 1, 3, i+1)
    sns.boxplot(y=df[feature])
    plt.title(f'Box plot of {feature}')
    plt.tight_layout()
plt.show()

print("Boxplots for all numerical features generated.")

In [ ]:
# =============================
# CORRELATION MATRIX
# =============================
correlation_matrix_X = df[num_features].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix_X, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Features')
plt.show()

# Combine X + y for target correlation
correlations_with_target = df.corr()['quality'].drop('quality')

print("\nCorrelations with target variable:")
print(correlations_with_target)

In [ ]:
# =============================
# QUALITY DISTRIBUTION (BAR + PIE)
# =============================
plt.figure(figsize=(15, 6))

# Bar chart
plt.subplot(1, 2, 1)
quality_counts = df['quality'].value_counts().sort_index()
plt.bar(quality_counts.index, quality_counts.values, edgecolor='black')
plt.title('Quality Rating Distribution')
plt.xlabel('Quality')
plt.ylabel('Count')

# Pie chart
plt.subplot(1, 2, 2)
plt.pie(
    quality_counts,
    autopct=lambda p: f'{p:.1f}%' if p > 1 else '',
    startangle=90,
    pctdistance=1.2,
    labeldistance=1.3,
    textprops={'fontsize': 12},
    wedgeprops={'edgecolor': 'white', 'linewidth': 1}
)
plt.legend(
    quality_counts.index,
    title="Quality Rating",
    loc="center left",
    bbox_to_anchor=(1, 0.5)
)
plt.title('Proportion of Quality Ratings')
plt.tight_layout()
plt.show()

In [ ]:

# =============================
# BOX PLOTS OF FEATURES VS TARGET
# =============================
key_features = df.columns.drop("quality")

plt.figure(figsize=(18, 12))
p = len(key_features)

for i, feature in enumerate(key_features):
    plt.subplot(round(p**0.5) + 1, round(p**0.5) + 1, i + 1)
    sns.boxplot(x=df["quality"], y=df[feature])
    plt.title(f'{feature} vs Quality')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

print("Boxplots showing relationship between features and quality generated.")


In [ ]:
# ---------------------------------------------------------------
# 2. PREPROCESSING (QUALITY BINARIZATION)
# ---------------------------------------------------------------

print("\n--- 2. PREPROCESSING ---")

# Convert quality → binary classes
bins = (2, 6.5, 8)
labels = ['Bad', 'Good']
data['quality'] = pd.cut(data['quality'], bins=bins, labels=labels)

lc = LabelEncoder()
data['quality'] = lc.fit_transform(data['quality'])

X = data.drop('quality', axis=1)
y = data['quality']


In [ ]:
# ---------------------------------------------------------------
# 2C. FEATURE IMPORTANCE (EXTRATREES)
# ---------------------------------------------------------------

extra = ExtraTreesClassifier(random_state=42)
extra_feat = extra.fit(X, y)
ex_data = pd.Series(extra_feat.feature_importances_, index=X.columns)

print("\nTop 5 Feature Importances:")
print(ex_data.nlargest(5))

# Feature importance plot
plt.figure(figsize=(10,5))
ex_data.sort_values().plot(kind='bar')
plt.title("Feature Importances (ExtraTreesClassifier)")
plt.xlabel("Features")
plt.ylabel("Importance Score")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# ---------------------------------------------------------------
# 2D. TRAIN–TEST SPLIT & STANDARDIZATION
# ---------------------------------------------------------------

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
# ---------------------------------------------------------------
# 3. BASELINE MODELS
# ---------------------------------------------------------------

print("\n--- 3. BASELINE MODELS ---")

models = [
    ("Logistic Regression", LogisticRegression(random_state=42, solver='liblinear')),
    ("KNN", KNeighborsClassifier()),
    ("Decision Tree", DecisionTreeClassifier(random_state=42)),
    ("SVC", SVC(random_state=42)),
    ("Random Forest", RandomForestClassifier(random_state=42)),
    ("Gradient Boosting", GradientBoostingClassifier(random_state=42)),
]

baseline_results = []

for name, model in models:
    print(f"\nTraining {name}...")

    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    baseline_results.append({
        'Model': name,
        'Accuracy': accuracy,
        'F1-Good': report['1']['f1-score']
    })

    print(f"Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))

baseline_df = pd.DataFrame(baseline_results).sort_values(by="F1-Good", ascending=False)
print("\nBASELINE SUMMARY:\n", baseline_df)

In [ ]:
# ---------------------------------------------------------------
# 4. HYPERPARAMETER TUNING
# ---------------------------------------------------------------

print("\n--- 4. HYPERPARAMETER TUNING (RandomizedSearchCV) ---")

TUNING_SCORING = "f1"
N_ITER_SEARCH = 15
CV_FOLDS = 5

models_to_tune = [

    ("Logistic Regression", LogisticRegression(random_state=42, solver="liblinear"), {
        "C": np.logspace(-3, 2, 10),
        "penalty": ["l1", "l2"]
    }),

    ("KNN", KNeighborsClassifier(), {
        "n_neighbors": np.arange(1, 25),
        "weights": ["uniform", "distance"]
    }),

    ("Decision Tree", DecisionTreeClassifier(random_state=42), {
        "max_depth": np.arange(2, 15),
        "min_samples_split": np.arange(2, 10),
        "min_samples_leaf": np.arange(1, 5)
    }),

    ("SVC", SVC(random_state=42), {
        "C": np.logspace(-1, 2, 10),
        "gamma": np.logspace(-3, 0, 10)
    }),

    ("Random Forest", RandomForestClassifier(random_state=42), {
        "n_estimators": [200, 300, 500],
        "max_depth": [5, 10, 20, None],
        "min_samples_leaf": [1, 2, 4]
    }),

    ("Gradient Boosting", GradientBoostingClassifier(random_state=42), {
        "n_estimators": [200, 300, 500],
        "learning_rate": [0.01, 0.05, 0.1],
        "max_depth": [3, 5, 8]
    }),
]

tuned_results = []
tuned_models = []

for name, model, grid in models_to_tune:

    print(f"\n🔍 Tuning {name}...")

    search = RandomizedSearchCV(
        model,
        grid,
        n_iter=N_ITER_SEARCH,
        cv=CV_FOLDS,
        scoring=TUNING_SCORING,
        random_state=42,
        n_jobs=-1
    )

    search.fit(x_train, y_train)
    best = search.best_estimator_

    tuned_models.append((name, best))
    print(f"Best params: {search.best_params_}")

In [ ]:

# ---------------------------------------------------------------
# 5. FINAL EVALUATION OF TUNED MODELS
# ---------------------------------------------------------------

print("\n--- 5. FINAL RESULTS (TUNED MODELS) ---")

for name, model in tuned_models:

    print(f"\nEvaluating Tuned {name}...")
    y_pred = model.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)

    tuned_results.append({
        "Model": name,
        "Accuracy": accuracy,
        "F1-Good": report["1"]["f1-score"]
    })

    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

tuned_df = pd.DataFrame(tuned_results).sort_values(by="F1-Good", ascending=False)
print("\nTUNED MODEL SUMMARY:\n", tuned_df)

# Plot Tuned Model F1 Scores
plt.figure(figsize=(9, 5))
sns.barplot(x="F1-Good", y="Model", data=tuned_df, palette="viridis")
plt.title("F1 Score (Good Class) - Tuned Models")
plt.show()

best_model = tuned_df.iloc[0]
print(f"\n🏆 Best Model: {best_model['Model']}  (F1 = {best_model['F1-Good']:.4f})")


In [ ]:
# import joblib
# model_filename = "best_wine_quality_model.pkl"
# joblib.dump(tuned_models[0][1], model_filename)